### This notebook is a demonstration of how to use our topic model to:

1. Obtain topics
2. Visualize topic transition heat maps
3. Reproduce topic transition chain/graphs



In [2]:
# import dependencies
import sys

import matplotlib
import matplotlib.pyplot as plt
import random
import torch
from torch import nn, optim
from torch.autograd import Function
import numpy as np
import os
import pickle
import data_analysis_utils
from data_analysis_utils import (prepare_text_for_lda, read_from_json_export, tokenize_doc_list, build_reduced_glove_dict,
convert_token_2_ids, read_from_json_export_by_story, text_to_topic, compute_topic_transition_matrix, filtere_off_unk_topics)
from collections import Counter
import dae_model
from dae_model import DictionaryAutoencoder
import time

#### GPU required